In [ ]:
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, GlobalAveragePooling1D, Dense
from keras.datasets import imdb
import numpy as np

# Constants for data preprocessing
max_length = 256  # Maximum length of the sequences
padding_type = 'post'  # Padding type for sequences shorter than the maximum length
vocab_size = 10000  # Size of the vocabulary used in the Embedding layer
embedding_dim=16  # Dimensionality of the embedding layer
hidden_units=16  # Number of hidden units in the LSTM layer

# Load the IMDB dataset
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=vocab_size)

# Helper function to preprocess data
def preprocess_data(data):
    return pad_sequences(data, maxlen=max_length, padding=padding_type)

# Preprocess the data
train_data = preprocess_data(train_data)
test_data = preprocess_data(test_data)

# Define the model architecture
def build_model(vocab_size, embedding_dim, hidden_units):
    model = Sequential([
        Embedding(vocab_size, embedding_dim, input_length=max_length),
        GlobalAveragePooling1D(),
        Dense(hidden_units, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    return model

# Build and compile the model
model = build_model(vocab_size, embedding_dim, hidden_units)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# Train and evaluate the model
history = model.fit(train_data, train_labels, epochs=10, batch_size=32, validation_data=(test_data, test_labels), verbose=2)
test_loss, test_acc = model.evaluate(test_data, test_labels, verbose=2)
print(f"Test Accuracy: {test_acc}, Test Loss: {test_loss}")

# Decode review function
word_index = imdb.get_word_index()
def decode_review(encoded_review):
    reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in encoded_review if i >= 3])

# Display incorrect predictions
def display_incorrect_predictions(test_data, test_labels, predictions, num_examples=3):
    predicted_classes = (predictions > 0.5).astype(int)
    incorrect_indices = np.where(predicted_classes.flatten() != test_labels)[0]
    for i, idx in enumerate(incorrect_indices[:num_examples]):
        print(f"Incorrect Prediction {i+1}:")
        print(f"Review: {decode_review(test_data[idx])}")
        print(f"Actual Sentiment: {'Positive' if test_labels[idx] == 1 else 'Negative'}")
        print(f"Predicted Sentiment: {'Positive' if predicted_classes[idx][0] == 1 else 'Negative'}")
        print("--------------------------------------------------------------------------------\n")

predictions = model.predict(test_data)
display_incorrect_predictions(test_data, test_labels, predictions)

# Predict sentiments for sample reviews and display them
def predict_and_display_reviews(reviews):
    sequences = [[word_index.get(word, 2) for word in review.lower().split()] for review in reviews]
    padded_sequences = preprocess_data(sequences)
    sample_predictions = model.predict(padded_sequences)
    sample_predicted_classes = (sample_predictions > 0.5).astype(int)
    for i, review in enumerate(reviews):
        print(f"Review {i+1}: {review}")
        print(f'Predicted Score: {sample_predictions[i]}')
        print(f"Predicted Sentiment: {'Positive' if sample_predicted_classes[i][0] == 1 else 'Negative'}")
        print("--------------------------------------------------------------------------------\n")

# Sample movie reviews
reviews = [
    "This movie was an excellent portrayal of character development and had stellar acting.",
    "I found the movie to be predictable with a lackluster script.",
    "The cinematography was magnificent, and the pacing was perfect. Highly recommend watching.",
    "It was a terrible movie that wasted two hours of my life. The plot made no sense.",
    "An absolute masterpiece, with a gripping story and profound performances."
]

predict_and_display_reviews(reviews)


Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 256, 32)           320000    
                                                                 
 global_average_pooling1d_8  (None, 32)                0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_21 (Dense)            (None, 32)                1056      
                                                                 
 dense_22 (Dense)            (None, 1)                 33        
                                                                 
Total params: 321089 (1.22 MB)
Trainable params: 321089 (1.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
782/782 - 7s - loss: 0.4471 - accuracy: 0.8054 - val_loss: 0.3108 - val_a

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.datasets import imdb
import numpy as np

# Parameters
vocab_size = 10000
max_length = 256
embedding_dim = 16
lstm_units = 64

# Load IMDB dataset
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=vocab_size)

# Preprocess data: Pad sequences to ensure uniform input size
train_data = pad_sequences(train_data, maxlen=max_length, padding='post')
test_data = pad_sequences(test_data, maxlen=max_length, padding='post')

# Define a simple Attention Layer
class Attention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(Attention, self).__init__()
        self.W = tf.keras.layers.Dense(units)
        self.U = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, inputs):
        # inputs: [batch_size, time_steps, input_dim]
        score = tf.nn.tanh(self.W(inputs) + self.U(inputs))
        attention_weights = tf.nn.softmax(self.V(score), axis=1)
        context_vector = attention_weights * inputs
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, attention_weights

# Build the model
input_layer = Input(shape=(max_length,))
embed = Embedding(vocab_size, embedding_dim, input_length=max_length)(input_layer)
lstm = LSTM(lstm_units, return_sequences=True)(embed)
attention_output, attention_weights = Attention(lstm_units)(lstm)
dense = Dense(64, activation='relu')(attention_output)
output_layer = Dense(1, activation='sigmoid')(dense)

model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()

# Train the model
model.fit(train_data, train_labels, epochs=5, batch_size=32, validation_data=(test_data, test_labels), verbose=2)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256)]             0         
                                                                 
 embedding_4 (Embedding)     (None, 256, 16)           160000    
                                                                 
 lstm (LSTM)                 (None, 256, 64)           20736     
                                                                 
 attention (Attention)       ((None, 64),              8385      
                              (None, 256, 1))                    
                                                                 
 dense_11 (Dense)            (None, 64)                4160      
                                                                 
 dense_12 (Dense)            (None, 1)                 65        
                                                             

# Gemini

In [ ]:
import numpy as np
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Bidirectional, Dense
from keras import backend as K
from keras.layers import Layer
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix

# Custom Dot-Product Attention Layer
class SimpleAttention(Layer):
    def __init__(self, **kwargs):
        super(SimpleAttention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3
        self.W = self.add_weight(name='attention_weight',
                                 shape=(input_shape[-1], 1),
                                 initializer='random_normal',
                                 trainable=True)
        super(SimpleAttention, self).build(input_shape)

    def call(self, x):
        e = K.tanh(K.dot(x, self.W))
        a = K.softmax(e, axis=1)
        output = x * a
        return output

# Hyperparameters
vocab_size = 5000
embedding_dim = 32
max_length = 200
lstm_units = 64

# Load IMDB Dataset
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)

# Preprocessing
X_train = sequence.pad_sequences(X_train, maxlen=max_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_length)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Build the Model
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
model.add(Bidirectional(LSTM(lstm_units, return_sequences=True)))
model.add(SimpleAttention())
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train
model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.2)

# Evaluation
loss, accuracy = model.evaluate(X_test, y_test)
print('Accuracy:', accuracy)

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

cm = confusion_matrix(y_true, y_pred_classes)
print(cm)

# Function to decode the reviews
word_index = imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in text])

# Incorrect Predictions Examples
incorrect_indices = np.nonzero(y_pred_classes != y_true)[0]
for i in incorrect_indices[:5]:
    text = X_test[i]
    decoded_text = decode_review(text)
    print('Original Review:', decoded_text)
    print('Predicted Sentiment:', 'Positive' if y_pred_classes[i] == 1 else 'Negative')
    print('True Sentiment:', 'Positive' if y_true[i] == 1 else 'Negative')
    print('---')


Epoch 1/5


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1151, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1209, in compute_loss
        return self.compiled_loss(
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/compile_utils.py", line 277, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    